<a href="https://colab.research.google.com/github/Jakelinecs/Tareas-Machine-Learning/blob/main/N29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import zipfile
import os

# Specify the path to your zip file
zip_file_path = 'train.zip' # Replace with the actual path to your zip file

# Specify the directory where you want to extract the contents
extraction_dir = '/content/train' # You can change this to your desired directory

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_dir, exist_ok=True)

# Open the zip file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extraction_dir)

print(f"Successfully extracted '{zip_file_path}' to '{extraction_dir}'")

Successfully extracted 'train.zip' to '/content/train'


In [14]:
# coding: utf-8
import numpy as np
import pandas as pd
import os
from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split

# --- CONFIGURACIÓN DE RUTAS Y PARÁMETROS ---

# Directorio base donde se encuentran las carpetas 'images' y 'masks' del TGS Salt Challenge
# ¡ATENCIÓN!: Reemplace esta ruta con la ubicación real de sus datos de Kaggle
DATA_DIR = './train/'

# Tamaño de imagen de entrada original (TGS)
IMG_SIZE_ORIG = 101
# Tamaño objetivo para U-Net (idealmente potencia de 2, como 128)
IMG_SIZE_TARGET = 128
# Número de canales de entrada (Imágenes en escala de grises = 1)
IMG_CHANNELS = 1

# --- GENERADOR DE DATOS DE KERAS ---

class TGSDataGenerator(Sequence):
    """
    Generador de datos personalizado de Keras para cargar imágenes y máscaras en batches.
    Aplica el redimensionamiento (padding de 101x101 a 128x128) y normalización.
    """
    def __init__(self, df, batch_size=16, shuffle=True, augment=False):
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment # Futura implementación de aumento de datos
        self.on_epoch_end()

    def __len__(self):
        """Número de batches por época"""
        return int(np.floor(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        """Mezcla los índices después de cada época si shuffle está activado"""
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        """Genera un batch de datos (X e Y)"""
        # Obtiene los IDs de las imágenes para el batch actual
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = self.df.iloc[indexes]['id'].tolist()

        # Genera los datos
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def __data_generation(self, list_IDs_temp):
        """Genera el contenido real del batch"""
        X = np.empty((self.batch_size, IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS), dtype=np.float32)
        y = np.empty((self.batch_size, IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS), dtype=np.float32)

        for i, id_name in enumerate(list_IDs_temp):
            # 1. Carga de la imagen (X)
            img = imread(os.path.join(DATA_DIR, 'images', id_name + '.png'))
            # Escala de grises (101, 101, 1). Se extrae el primer canal si se carga como RGB
            img = img[:, :, 0] if img.ndim == 3 else img

            # 2. Carga de la máscara (Y)
            mask = imread(os.path.join(DATA_DIR, 'masks', id_name + '.png'))
            mask = mask[:, :, 0] if mask.ndim == 3 else mask

            # 3. Redimensionamiento y Padding (101x101 -> 128x128)
            # Nota: resize aplica internamente el padding necesario si el tamaño de salida es mayor.
            # Usaremos resize para escalar y luego expandir la dimensión de canal.

            # Redimensionar la imagen y la máscara al tamaño objetivo (128x128)
            img_resized = resize(img, (IMG_SIZE_TARGET, IMG_SIZE_TARGET), mode='constant', preserve_range=True)
            mask_resized = resize(mask, (IMG_SIZE_TARGET, IMG_SIZE_TARGET), mode='constant', preserve_range=True)

            # 4. Normalización
            # Las imágenes se normalizan a [0, 1]
            X[i,] = (img_resized / 255.0).reshape(IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS)
            # La máscara (etiquetas binarias) se normaliza a [0, 1]
            y[i,] = (mask_resized / 255.0).reshape(IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS)


        return X, y

# --- FUNCIÓN PRINCIPAL DE CARGA DE DATOS ---

def load_tgs_data(data_dir=DATA_DIR, batch_size=16, test_size=0.2, val_size=0.2):
    """
    Carga los IDs, divide el conjunto de datos y crea los generadores.

    Returns:
        train_generator, val_generator, test_generator
    """
    print(f"Buscando datos en: {data_dir}")
    # Cargar IDs de imágenes
    all_files = [f.split('.')[0] for f in os.listdir(os.path.join(data_dir, 'images')) if f.endswith('.png')]
    df_ids = pd.DataFrame({'id': all_files})

    if len(df_ids) == 0:
        print("ERROR: No se encontraron archivos de imagen. Verifique la ruta DATA_DIR.")
        return None, None, None

    # División: Train y Temp (para Val y Test)
    train_df, temp_df = train_test_split(df_ids, test_size=(test_size + val_size), random_state=42)

    # División: Val y Test
    val_df, test_df = train_test_split(temp_df, test_size=test_size/(test_size + val_size), random_state=42)

    print(f"Total imágenes encontradas: {len(df_ids)}")
    print(f"Train samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    print(f"Test samples: {len(test_df)}")

    # Creación de generadores
    train_gen = TGSDataGenerator(train_df, batch_size=batch_size, shuffle=True)
    val_gen = TGSDataGenerator(val_df, batch_size=batch_size, shuffle=False)
    test_gen = TGSDataGenerator(test_df, batch_size=batch_size, shuffle=False)

    return train_gen, val_gen, test_gen

# --- EJEMPLO DE USO (SOLO PARA PRUEBA) ---
if __name__ == '__main__':
    # Nota: Este bloque de código solo funcionará si los datos están presentes en DATA_DIR
    print("--- Prueba de Carga de Datos ---")
    print(f"Buscando datos en: {DATA_DIR}")

    # Verificar si el directorio existe
    if os.path.exists(DATA_DIR):
        print("✓ Directorio de datos encontrado")

        # Verificar subdirectorios
        images_dir = os.path.join(DATA_DIR, 'images')
        masks_dir = os.path.join(DATA_DIR, 'masks')

        if os.path.exists(images_dir):
            print("✓ Directorio 'images' encontrado")
            image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
            print(f"  - {len(image_files)} archivos de imagen encontrados")
        else:
            print("✗ Directorio 'images' no encontrado")

        if os.path.exists(masks_dir):
            print("✓ Directorio 'masks' encontrado")
            mask_files = [f for f in os.listdir(masks_dir) if f.endswith('.png')]
            print(f"  - {len(mask_files)} archivos de máscara encontrados")
        else:
            print("✗ Directorio 'masks' no encontrado")

        # Si ambos directorios existen, intentar cargar los datos
        if os.path.exists(images_dir) and os.path.exists(masks_dir):
            print("\n--- Intentando cargar datos ---")
            try:
                train_gen, val_gen, test_gen = load_tgs_data(batch_size=16)

                if train_gen:
                    print("✓ Generadores creados exitosamente")
                    X_batch, y_batch = train_gen.__getitem__(0)
                    print(f"\nShape de X (Imágenes): {X_batch.shape}")
                    print(f"Shape de y (Máscaras): {y_batch.shape}")
                    print(f"Valores Máx/Mín de X: {X_batch.min():.2f} / {X_batch.max():.2f}")
                    print(f"Valores Máx/Mín de y: {y_batch.min():.2f} / {y_batch.max():.2f}")
                    print(f"Número de batches por época: {len(train_gen)}")
                else:
                    print("✗ No se pudieron crear los generadores")
            except Exception as e:
                print(f"✗ Error al cargar datos: {e}")
        else:
            print("\n⚠️  No se pueden cargar los datos porque faltan directorios")
    else:
        print("✗ Directorio de datos no encontrado")
        print(f"  Ruta configurada: {DATA_DIR}")
        print("  Para usar este script, descargue el dataset TGS Salt Challenge de Kaggle")
        print("  y ajuste la variable DATA_DIR en el script")

    print("\n--- Información del Script ---")
    print("Este script está diseñado para el TGS Salt Identification Challenge de Kaggle")
    print("Funcionalidades:")
    print("  - Carga imágenes y máscaras de segmentación")
    print("  - Redimensiona de 101x101 a 128x128 píxeles")
    print("  - Normaliza valores a [0,1]")
    print("  - Divide datos en train/validation/test")
    print("  - Genera batches para entrenamiento con Keras")


--- Prueba de Carga de Datos ---
Buscando datos en: ./train/
✓ Directorio de datos encontrado
✓ Directorio 'images' encontrado
  - 4000 archivos de imagen encontrados
✓ Directorio 'masks' encontrado
  - 4000 archivos de máscara encontrados

--- Intentando cargar datos ---
Buscando datos en: ./train/
Total imágenes encontradas: 4000
Train samples: 2400
Validation samples: 800
Test samples: 800
✓ Generadores creados exitosamente

Shape de X (Imágenes): (16, 128, 128, 1)
Shape de y (Máscaras): (16, 128, 128, 1)
Valores Máx/Mín de X: 0.00 / 1.00
Valores Máx/Mín de y: 0.00 / 257.00
Número de batches por época: 150

--- Información del Script ---
Este script está diseñado para el TGS Salt Identification Challenge de Kaggle
Funcionalidades:
  - Carga imágenes y máscaras de segmentación
  - Redimensiona de 101x101 a 128x128 píxeles
  - Normaliza valores a [0,1]
  - Divide datos en train/validation/test
  - Genera batches para entrenamiento con Keras
